In [12]:
import sys
import os
sys.path.append(os.path.abspath('..'))

from langchain_aws import BedrockChat
from langchain.chains import LLMChain, ConversationalRetrievalChain
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain_community.retrievers import AmazonKnowledgeBasesRetriever
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

from app.config import config

from dotenv import load_dotenv
load_dotenv()


True

# env init

In [15]:
kb_id=config["rags"]["CrossOver"]["knowledge_base_id"]
print("knowledge base id:", kb_id)
model_kwargs = {
    "temperature": 0.1,
    "top_p": 0.999,
    "top_k": 500,
    "max_tokens": 4096,
}
model_id = config["models"]["Claude 3 Sonnet"]["model_id"]
print("model id:", model_id)

# 創建向量存儲和檢索器
retriever = AmazonKnowledgeBasesRetriever(
    knowledge_base_id=kb_id,
    retrieval_config={"vectorSearchConfiguration": {"numberOfResults": 4}},
)

# 創建 LLM
llm = BedrockChat(model_id=model_id, model_kwargs=model_kwargs, streaming=True)

knowledge base id: RTVIA3VQNZ
model id: anthropic.claude-3-sonnet-20240229-v1:0


/Users/br/miniconda3/envs/bedrock/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `BedrockChat` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use ChatBedrock instead.
  warn_deprecated(


# rag with chat

In [5]:
template = '''Answer the question based only on the following context:
{context}

Question: {question}'''
prompt = ChatPromptTemplate.from_template(template)


chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    .assign(response = prompt | llm | StrOutputParser())
    .pick(["response", "context"])
)

# 示例：車主提問
question = "如何開啟和關閉 Gogoro 電動機車的系統電源？"
response = chain.invoke(question)
print(response['response'])

根據手冊內容,開啟和關閉 Gogoro 電動機車的系統電源方式有以下幾種:

1. 機械式鑰匙車種:
- 開啟系統電源並解鎖龍頭: 將鑰匙插入龍頭鎖孔,順時針轉動解鎖。
- 關閉系統電源並上鎖龍頭: 將鑰匙插入龍頭鎖孔,逆時針轉動上鎖。

2. 無線鑰匙車種:
- 使用 iQ System 無線智慧鑰匙、智慧鑰匙卡或 Gogoro Smart Coin:靠近感應區即可自動開啟/關閉系統電源並解鎖/上鎖龍頭。
- 使用手機作為遙控器:透過 Gogoro App 開啟/關閉系統電源並解鎖/上鎖龍頭。
- 使用手機作為免鑰匙智慧感應器:靠近車輛即可自動開啟系統電源並解鎖龍頭,離開一段距離後會自動關閉系統電源。

因此無論是使用機械鑰匙或無線智慧鑰匙系統,都可以方便地開啟和關閉 Gogoro 電動機車的系統電源。


In [6]:
retriever.invoke(question)

[Document(page_content='傳動皮帶或鏈條調整、檢查、清潔及潤滑作業過程中請務必確保車輛保持系統電源關閉狀態，嚴   禁在馬達已啟動狀態下作業，以免衣物或肢體被機件捲入發生危險。   若您不確定如何操作或實施以上項目，在過程中有疑慮或發現異常，請諮詢專業人員、聯繫   Gogoro 客服中心（0800-365-996），或至 Gogoro 服務中心尋求協助。   85        7.2 Gogoro CrossOver系列定期檢查與保養週期   當保養里程或時間屆至時，系統會透過 Gogoro® App 及 GoStation® 電池交換站主動提醒您   定期保養之訊息。您可以就近至 Gogoro 服務中心進行檢查與例行保養。   ● 新車交車後初次回廠檢查週期：交車後 1,000 公里或 2 個月（以先到者為準）。   ● 定期回廠檢查與保養週期：每 3,000 公里或每 6 個月（以先到者為準）。   保養里程   (單位：1,000 km) 1 4 7 10 13 16 19 22 25 28 31 34 37   保養時間   (單位：月) 2 6 12 18 24 30 36 42 48 54 60 66 72   各項開關作動 C C C C C C C C C C C C C 主、側腳架 / 坐墊扣座   / 煞車拉桿 / 後視鏡 / 鑰匙孔 (若有配備)   L L L L L L L L L L L L L   煞車油 C C C C C C R C C C C C R 齒輪油 R R R R R 輪胎 / 煞車皮 / 碟盤 C C C C C C C C C C C C C 冷卻系統 C C C C C C C C C C C C C   傳動系統 (鏈條) L / A / C   L / A / C   L / A / C   L / A / C   L / A / C   L / A / C   L / A / C', metadata={'location': {'s3Location': {'uri': 's3://llm-knowladgebase/CrossOver_br/Gogoro CrossOver 系列 Smartscooter® 智慧電動機車使用手冊(2023.12).pdf'}, 'type': '

# System prompt

In [48]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是Gogoro Smart Scooter的專家,你可以根據使用者使用的語言,用相同的語言來回答有關Smart Scooter的問題。使用者的Gogoro車種為{car_model}。根據{car_model}參數,從知識庫中搜尋對應該車種的文件或章節資訊。從相關文件或章節中整理出與問題最相關的資訊作為回答。如果問題與Gogoro Smart Scooter無關,你將禮貌地告知使用者你無法回答此類問題。根據知識庫的內容,你可以用{language}回答以下範圍的問題:安全注意事項, Gogoro Smartscooter簡介, Gogoro Network智慧電池簡介, GoStation 電池交換簡介, Gogoro App簡介, iQ System簡介, {car_model}各部位名稱, 左把手, 右把手, 儀表板, {car_model}系列座墊及置物箱操作方式, 機械式鑰匙車種, 無線鑰匙車種, 預估剩餘電量可行駛里程, 取車及架車, 啟動及關閉馬達, 前進及後退, 減速及停止, 動力模式, 進階功能, 在 GoStation 電池交換站交換電池, 對 Gogoro Network 智慧電池充電, 下載及安裝 Gogoro App, 將手機與 Smartscooter 智慧電動機車配對連線, 日常清潔與維護, Gogoro {car_model}系列定期檢查與保養週期, 服務據點, NCC 國家通訊傳播委員會, 行政院環保署, 經濟部能源局。如果問題超出上述範圍,你將禮貌地用{language}告知使用者此問題與Gogoro Smart Scooter無關,你無法回答。請提供約100-300字的詳細答覆,使用簡潔明確的方式用{language}回答問題,並根據問題補充任何其他重要的相關資訊。"),
    ("human", "{user_input}")
])

prompt.format_messages(car_model="JEGO", language="繁體中文", user_input="測試")

[SystemMessage(content='你是Gogoro Smart Scooter的專家,你可以根據使用者使用的語言,用相同的語言來回答有關Smart Scooter的問題。使用者的Gogoro車種為JEGO。根據JEGO參數,從知識庫中搜尋對應該車種的文件或章節資訊。從相關文件或章節中整理出與問題最相關的資訊作為回答。如果問題與Gogoro Smart Scooter無關,你將禮貌地告知使用者你無法回答此類問題。根據知識庫的內容,你可以用繁體中文回答以下範圍的問題:安全注意事項, Gogoro Smartscooter簡介, Gogoro Network智慧電池簡介, GoStation 電池交換簡介, Gogoro App簡介, iQ System簡介, JEGO各部位名稱, 左把手, 右把手, 儀表板, JEGO系列座墊及置物箱操作方式, 機械式鑰匙車種, 無線鑰匙車種, 預估剩餘電量可行駛里程, 取車及架車, 啟動及關閉馬達, 前進及後退, 減速及停止, 動力模式, 進階功能, 在 GoStation 電池交換站交換電池, 對 Gogoro Network 智慧電池充電, 下載及安裝 Gogoro App, 將手機與 Smartscooter 智慧電動機車配對連線, 日常清潔與維護, Gogoro JEGO系列定期檢查與保養週期, 服務據點, NCC 國家通訊傳播委員會, 行政院環保署, 經濟部能源局。如果問題超出上述範圍,你將禮貌地用繁體中文告知使用者此問題與Gogoro Smart Scooter無關,你無法回答。請提供約100-300字的詳細答覆,使用簡潔明確的方式用繁體中文回答問題,並根據問題補充任何其他重要的相關資訊。'),
 HumanMessage(content='測試')]

In [55]:
from langchain.chains import create_retrieval_chain
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="user_input"
)

chainWithMemory = LLMChain(llm=llm, prompt=prompt, memory=memory)
retrieval_chain = create_retrieval_chain(retriever, chainWithMemory)
response = retrieval_chain.invoke({
    "car_model": "JEGO", 
    "language": "繁體中文", #["English", "繁體中文"]
    "user_input": "請問你該服務我哪種車款？" 
})
print(response)

KeyError: 'input'